In [86]:
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import datetime
from utils.extract import db_connection, download_data
import json

# Define the instruments to download. We would like to see Apple, Microsoft and the S&P500 index.

lista_tickers = pd.read_csv('tickers.csv')[['Ticker']]

now = datetime.datetime.now()

# We would like all available data from 01/01/2000 until 12/31/2016.
start_date = '2000-01-01'
end_date = '{}-{}-{}'.format(now.year, now.month, now.day)

with open('creds.txt', encoding='utf-8') as data_file:
    creds = json.loads(data_file.read())

In [99]:
def create_new_stock_table(table_name, creds):
    """
    Hace una nueva tabla de stocks según el ticker que le pongas
    
    Args:
        table_name(str): nombre del ticker
        conn(connection object): objeto de connección a la base de datos
    """
    conn = db_connection(creds)
    query = """CREATE TABLE IF NOT EXISTS {} (id SERIAL,
    date date,
    high float,
    low float,
    open float,
    close float,
    volume float, 
    adj_close float, PRIMARY KEY (id));""".format(table_name)

def update_in_db(df, table_name, creds):
    
    conn = db_connection(creds)
    data = [str(i).replace('[','(').replace(']',')') for i in df.as_matrix()]
    data = str(data).replace('[','(').replace(']',')').replace("'","")
    query = """INSERT INTO {} (date, high, low, open, close, volume, adj_close)
    VALUES{} 
    ON CONFLICT (date) 
    DO
    UPDATE
    SET email = EXCLUDED.email || ';' || customers.email;""".format(table_name, data)
    print(query)
    res = download_data(conn, query)
    
    return res

In [100]:
# User pandas_reader.data.DataReader to load the desired data. As simple as that.

for ticker in lista_tickers['Ticker']:
    print("Se guardó: {}".format(ticker))
    panel_data = data.DataReader(ticker, 'yahoo', start_date, end_date)
    table_name = ticker.replace('.','_')
    create_new_stock_table(table_name, creds)
    update_in_db(panel_data, table_name, creds)
    

Se guardó: XLK.MX
INSERT INTO XLK_MX (date, high, low, open, close, volume, adj_close)
    VALUES((265.70001221 265.70001221 265.70001221 265.70001221   0.\n 265.70001221), (265.35998535 265.35998535 265.35998535 265.35998535   0.\n 265.35998535), (271.3999939 271.3999939 271.3999939 271.3999939   0.        271.3999939), (272.3999939  271.73001099 271.73001099 272.3999939    0.\n 272.3999939 ), (274.10998535 274.10998535 274.10998535 274.10998535   0.\n 274.10998535), (277.5 277.5 277.5 277.5   0.  277.5), (281. 281. 281. 281.   0. 281.), (289.3999939 289.3999939 289.3999939 289.3999939   0.        289.3999939), (289.75 289.75 289.75 289.75   0.   289.75), (282. 282. 282. 282.   0. 282.), (283.58999634 283.58999634 283.58999634 283.58999634   0.\n 283.58999634), (287.26000977 287.26000977 287.26000977 287.26000977   0.\n 287.26000977), (290.5 290.5 290.5 290.5   0.  290.5), (290.75 290.75 290.75 290.75   0.   290.75), (300. 300. 300. 300.   0. 300.), (299.79998779 299.79998779 299.7999

ProgrammingError: syntax error at or near "265.70001221"
LINE 2:     VALUES((265.70001221 265.70001221 265.70001221 265.70001...
                                 ^


In [59]:
panel_data = data.DataReader('XLK.MX', 'yahoo', start_date, end_date)

In [73]:
panel_data.iloc[[0]]

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2007-02-22,265.700012,265.700012,265.700012,265.700012,0.0,265.700012
